In [1]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import nltk, string
from re import sub
from decimal import Decimal
from collections import defaultdict
from statistics import median
from sklearn import preprocessing
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import seaborn as sns
import warnings
warnings.simplefilter('ignore')

In [2]:
airbnb = pd.read_csv('./airbnb_data.csv')
airbnb_sub = pd.read_csv('./airbnb_submission.csv')

In [3]:
airbnb = airbnb.drop(columns=['description', 'name', 'thumbnail_url','neighbourhood', 'zipcode'])
airbnb_sub = airbnb_sub.drop(columns=['description', 'name', 'thumbnail_url','neighbourhood', 'zipcode'])

In [4]:
from datetime import datetime
def conver_time(df):
    df=datetime.strptime(df, '%Y-%m-%d').date()
    return df.toordinal()

In [5]:
airbnb.first_review = airbnb.first_review.apply(conver_time)
airbnb_sub.first_review = airbnb_sub.first_review.apply(conver_time)
airbnb.last_review = airbnb.last_review.apply(conver_time)
airbnb_sub.last_review = airbnb_sub.last_review.apply(conver_time)
airbnb.host_since = airbnb.host_since.apply(conver_time)
airbnb_sub.host_since = airbnb_sub.host_since.apply(conver_time)

In [6]:
airbnb.cleaning_fee = airbnb.cleaning_fee.astype(int)
airbnb.host_has_profile_pic = airbnb.host_has_profile_pic.astype(bool)
airbnb.host_identity_verified = airbnb.host_identity_verified.astype(bool)
airbnb.instant_bookable = airbnb.instant_bookable.astype(bool)
airbnb.cancellation_policy = airbnb.cancellation_policy.astype(bool)
airbnb.host_has_profile_pic = airbnb.host_has_profile_pic.astype(int)
airbnb.host_identity_verified = airbnb.host_identity_verified.astype(int)
airbnb.instant_bookable = airbnb.instant_bookable.astype(int)
airbnb.cancellation_policy = airbnb.cancellation_policy.astype(int)
airbnb.host_response_rate = airbnb.host_response_rate.astype(bool)
airbnb.host_response_rate = airbnb.host_response_rate.astype(int)

In [7]:
airbnb_sub.cleaning_fee = airbnb_sub.cleaning_fee.astype(int)
airbnb_sub.host_has_profile_pic = airbnb_sub.host_has_profile_pic.astype(bool)
airbnb_sub.host_identity_verified = airbnb_sub.host_identity_verified.astype(bool)
airbnb_sub.instant_bookable = airbnb_sub.instant_bookable.astype(bool)
airbnb_sub.cancellation_policy = airbnb_sub.cancellation_policy.astype(bool)
airbnb_sub.host_has_profile_pic = airbnb_sub.host_has_profile_pic.astype(int)
airbnb_sub.host_identity_verified = airbnb_sub.host_identity_verified.astype(int)
airbnb_sub.instant_bookable = airbnb_sub.instant_bookable.astype(int)
airbnb_sub.cancellation_policy = airbnb_sub.cancellation_policy.astype(int)
airbnb_sub.host_response_rate = airbnb_sub.host_response_rate.astype(bool)
airbnb_sub.host_response_rate = airbnb_sub.host_response_rate.astype(int)

In [8]:
airbnb['amenities'] = airbnb['amenities'].str.replace('{','')
airbnb['amenities'] = airbnb['amenities'].str.replace('}','')
airbnb['amenities'] = airbnb['amenities'].str.replace('"','') 

In [9]:
airbnb_sub['amenities'] = airbnb_sub['amenities'].str.replace('{','')
airbnb_sub['amenities'] = airbnb_sub['amenities'].str.replace('}','')
airbnb_sub['amenities'] = airbnb_sub['amenities'].str.replace('"','') 

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer =  CountVectorizer(tokenizer=lambda x: x.split(','))
amenities = count_vectorizer.fit_transform(airbnb['amenities'])
df_amenities = pd.DataFrame(amenities.toarray(), columns=count_vectorizer.get_feature_names())
df_amenities = df_amenities.drop('',1,inplace=False)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer_sub = CountVectorizer(tokenizer=lambda x: x.split(','))
amenities_sub = count_vectorizer_sub.fit_transform(airbnb_sub['amenities'])
df_amenities_sub = pd.DataFrame(amenities_sub.toarray(), columns=count_vectorizer_sub.get_feature_names())
df_amenities_sub = df_amenities_sub.drop('',1,inplace=False)

In [12]:
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28877 entries, 0 to 28876
Data columns (total 23 columns):
property_type             28877 non-null object
room_type                 28877 non-null object
amenities                 28877 non-null object
accommodates              28877 non-null int64
bathrooms                 28877 non-null float64
bed_type                  28877 non-null object
cancellation_policy       28877 non-null int64
cleaning_fee              28877 non-null int64
city                      28877 non-null object
first_review              28877 non-null int64
host_has_profile_pic      28877 non-null int64
host_identity_verified    28877 non-null int64
host_response_rate        28877 non-null int64
host_since                28877 non-null int64
instant_bookable          28877 non-null int64
last_review               28877 non-null int64
latitude                  28877 non-null float64
longitude                 28877 non-null float64
number_of_reviews         28877 no

In [13]:
airbnb_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9625 entries, 0 to 9624
Data columns (total 23 columns):
id                        9625 non-null int64
property_type             9625 non-null object
room_type                 9625 non-null object
amenities                 9625 non-null object
accommodates              9625 non-null int64
bathrooms                 9625 non-null float64
bed_type                  9625 non-null object
cancellation_policy       9625 non-null int64
cleaning_fee              9625 non-null int64
city                      9625 non-null object
first_review              9625 non-null int64
host_has_profile_pic      9625 non-null int64
host_identity_verified    9625 non-null int64
host_response_rate        9625 non-null int64
host_since                9625 non-null int64
instant_bookable          9625 non-null int64
last_review               9625 non-null int64
latitude                  9625 non-null float64
longitude                 9625 non-null float64
number_o

In [14]:
airbnb2 = pd.concat([airbnb, df_amenities], axis=1, join='inner')

In [15]:
airbnb2_sub = pd.concat([airbnb_sub, df_amenities_sub], axis=1, join='inner')

In [16]:
airbnb2.shape

(28877, 152)

In [17]:
airbnb2_sub.shape

(9625, 147)

In [18]:
airbnb_numeric = airbnb._get_numeric_data().copy()
airbnb_sub_numeric = airbnb_sub._get_numeric_data().copy()

In [19]:
#def createOneHotVector(categories):
 #   categoryOneHotVector = dict()
  #  index = 0
   # for category in categories:
    #    oneHotVector = []
     #   for i in range(len(categories)):
      #      oneHotVector.append(1 if i == index else 0)
       # index += 1
        #categoryOneHotVector[category] = oneHotVector
    
  #  return categoryOneHotVector

In [20]:
#airbnb.roomTypeOneHotVector = createOneHotVector(airbnb.room_type.unique())
#airbnb.bedTypeOneHotVector = createOneHotVector(airbnb.bed_type.unique())
#airbnb_sub.roomTypeOneHotVector = createOneHotVector(airbnb_sub.room_type.unique())
#airbnb_sub.bedTypeOneHotVector = createOneHotVector(airbnb_sub.bed_type.unique())

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor as RFR
X=airbnb_numeric.drop('price', axis=1)
y=airbnb_numeric.price

In [22]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2)

In [23]:
rf=RFR(n_estimators=15, min_samples_leaf=3)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=3, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=15,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [24]:
train_score=rf.score(X_train, y_train)
test_score=rf.score(X_test, y_test)
print (train_score, test_score)

0.8716911291008422 0.6271801964791388


In [25]:
y_pred = rf.predict(airbnb_sub_numeric.drop('id', axis=1))

In [26]:
new_df = pd.DataFrame(columns=['id', 'price'])

In [27]:
new_df.id = airbnb_sub_numeric.id

In [28]:
new_df.price = y_pred

In [29]:
new_df.to_csv('firsttry.csv', index=False)

In [30]:
#lasso=Lasso()
#lasso.fit(X_train, y_train)

In [31]:
#X_train.shape

In [32]:
#y_pred=lasso.predict(airbnb_sub_numeric.drop('id', axis=1))

In [33]:
#lasso.score(X_test, y_test)

In [34]:
#from sklearn.svm import SVR
#svr=SVR()
#svr.fit(X_train, y_train)

In [35]:
#train_score=svr.score(X_train, y_train)
#test_score=svr.score(X_test, y_test)
#print (train_score, test_score)

In [36]:
#from sklearn.tree import ExtraTreeRegressor as ETR
#etr=ETR()
#etr.fit(X_train, y_train)

In [37]:
#train_score=etr.score(X_train, y_train)
#test_score=etr.score(X_test, y_test)
#print (train_score, test_score)